# Load libraries

In [49]:
library(Seurat)
library(Signac)
library(Matrix)
library(readr)
library(ggplot2)
library(data.table)
library(GenomicRanges)
library(dplyr)
library(rtracklayer)

# Step 1: Load sparse matrix and convert to full matrix

Use the dataset: "PBMCs 3k cells from a healthy donor"
* Use the material provided as part of the exam.

Matrix conversion
* Use the Matrix R package to convert the sparse matrix into a full matrix.
* Save the result as a data.table object.


In [3]:
data_path <- "Data/matrix/"

In [4]:
matrix <- readMM(file = paste0(data_path, "matrix.mtx.gz"))
features <- read_tsv(file = paste0(data_path, "features.tsv.gz"), col_names = F, show_col_types = F)
barcodes <- read_tsv(file = paste0(data_path, "barcodes.tsv.gz"), col_names = F, show_col_types = F) 

In [5]:
colnames(features) <- c("id", "name", "type", "chr", "start", "end")
head(features)

id,name,type,chr,start,end
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
ENSG00000243485,MIR1302-2HG,Gene Expression,chr1,29553,30267
ENSG00000237613,FAM138A,Gene Expression,chr1,36080,36081
ENSG00000186092,OR4F5,Gene Expression,chr1,65418,69055
ENSG00000238009,AL627309.1,Gene Expression,chr1,120931,133723
ENSG00000239945,AL627309.3,Gene Expression,chr1,91104,91105
ENSG00000239906,AL627309.2,Gene Expression,chr1,140338,140339


In [6]:
sum(features$type == "Gene Expression")

[1] 36601

In [56]:
table(features$type)
table(features$chr)


Gene Expression           Peaks 
          36601           81156 


GL000009.2 GL000194.1 GL000195.1 GL000205.2 GL000213.1 GL000218.1 GL000219.1 
         2          9          5          8          1          2         12 
KI270711.1 KI270713.1 KI270721.1 KI270726.1 KI270727.1 KI270728.1 KI270731.1 
         4          6          2          4          5          9          1 
KI270734.1       chr1      chr10      chr11      chr12      chr13      chr14 
         4      11430       5158       6098       6124       2598       4241 
     chr15      chr16      chr17      chr18      chr19       chr2      chr20 
      3970       4539       6425       2277       5959       8828       3315 
     chr21      chr22       chr3       chr4       chr5       chr6       chr7 
      1579       2784       6741       4674       5729       6802       5557 
      chr8       chr9       chrX       chrY 
      4730       4594       3393        125 

In [8]:
nrow(matrix)
nrow(features)
ncol(matrix)
nrow(barcodes)

[1] 117757

[1] 117757

[1] 3009

[1] 3009

In [9]:
colnames(matrix) <- barcodes$X1
rownames(matrix) <- features$id
matrix <- as.matrix(matrix)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 2.6 GiB”


In [10]:
dt <- as.data.table(matrix, row.names = rownames(matrix))
dt[, id := rownames(matrix)]
setcolorder(dt, c("id", setdiff(names(dt), "id")))
class(dt)
nrow(dt)
ncol(dt)
dt[1:10,1:10]

[1] "data.table" "data.frame"

[1] 117757

[1] 3010

id,AAACAGCCAACAGGTG-1,AAACATGCAACAACAA-1,AAACATGCACCTGGTG-1,AAACCAACACAGCCTG-1,AAACCAACAGCAAGAT-1,AAACCAACATTGCGAC-1,AAACCGAAGCACAGCC-1,AAACCGCGTAATTAGC-1,AAACCGCGTCACGAAC-1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000243485,0,0,0,0,0,0,0,0,0
ENSG00000237613,0,0,0,0,0,0,0,0,0
ENSG00000186092,0,0,0,0,0,0,0,0,0
ENSG00000238009,0,0,0,0,0,0,0,0,0
ENSG00000239945,0,0,0,0,0,0,0,0,0
ENSG00000239906,0,0,0,0,0,0,0,0,0
ENSG00000241860,0,0,0,0,0,0,0,0,0
ENSG00000241599,0,0,0,0,0,0,0,0,0
ENSG00000286448,0,0,0,0,0,0,0,0,0


# Step 2: Split gene expression and ATAC-seq data

From the data.table object, separate:
* Gene expression data (rows labelled with Ensembl gene IDs, e.g., ENSG00000243485)
* ATAC-seq peak data (rows labelled with genomic coordinates, e.g., chrN:NNNN-NNNN)


In [12]:
dt_genes <- dt[grepl("ENSG", dt$id)]
dt_atac <- rbind(dt[grepl("chr", dt$id)],dt[!grepl("ENSG|chr", dt$id)])

In [13]:
nrow(dt)

[1] 117757

In [14]:
nrow(dt_genes)

[1] 36601

In [15]:
nrow(dt_atac)

[1] 81156

In [16]:
nrow(dt_genes) + nrow(dt_atac)

[1] 117757

In [17]:
a <- grepl("ENSG|chr", dt$id, value = TRUE, invert = TRUE)

ERROR: Error in grepl("ENSG|chr", dt$id, value = TRUE, invert = TRUE): unused arguments (value = TRUE, invert = TRUE)


In [18]:
(features_dt[!grepl("ENSG|chr", features_dt$id)])

ERROR: Error: object 'features_dt' not found


In [19]:
(dt[!grepl("ENSG|chr", dt$id)])

id,AAACAGCCAACAGGTG-1,AAACATGCAACAACAA-1,AAACATGCACCTGGTG-1,AAACCAACACAGCCTG-1,AAACCAACAGCAAGAT-1,AAACCAACATTGCGAC-1,AAACCGAAGCACAGCC-1,AAACCGCGTAATTAGC-1,AAACCGCGTCACGAAC-1,⋯,TTTGGTGCATGAGCAG-1,TTTGTCCCAGCTTAGC-1,TTTGTCCCATAATCGT-1,TTTGTCTAGCATGTTA-1,TTTGTCTAGGAGGACT-1,TTTGTCTAGTCTATGA-1,TTTGTGGCAGCACGAA-1,TTTGTGGCATCGCTCC-1,TTTGTGTTCACTTCAT-1,TTTGTGTTCATGCGTG-1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
KI270728.1:232228-233172,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
KI270728.1:1791270-1791983,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
KI270728.1:1792102-1792511,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
KI270727.1:52087-52965,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,2,0
GL000009.2:113930-114840,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GL000194.1:27950-28827,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GL000194.1:55758-56625,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GL000194.1:59549-60358,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GL000194.1:71564-72412,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


# Step 3: Summarize Data

For each dataset (expression and peaks), compute the column-wise sum to produce:
* A single vector of total expression per gene
* A single vector of total chromatin accessibility per peak region

In [20]:
if (!requireNamespace("dplyr", quietly = TRUE))
    install.packages("dplyr")
library(dplyr)

In [21]:
genes_summary <- dt_genes[, lapply(.SD, sum), .SDcols = -"id"] %>% as.numeric()

In [102]:
names(genes_summary) <- dt_genes$id

ERROR: Error in names(genes_summary) <- dt_genes$id: 'names' attribute [36601] must be the same length as the vector [3009]


In [109]:
nrow(dt_genes)

[1] 36601

In [103]:
length(genes_summary)

[1] 3009

In [104]:
genes_summary[1] == sum(dt[,2])

[1] FALSE

In [106]:
sum(dt[,2])

[1] 13129

In [108]:
(dt[1,-1])

AAACAGCCAACAGGTG-1,AAACATGCAACAACAA-1,AAACATGCACCTGGTG-1,AAACCAACACAGCCTG-1,AAACCAACAGCAAGAT-1,AAACCAACATTGCGAC-1,AAACCGAAGCACAGCC-1,AAACCGCGTAATTAGC-1,AAACCGCGTCACGAAC-1,AAACCGGCAGAGGGAG-1,⋯,TTTGGTGCATGAGCAG-1,TTTGTCCCAGCTTAGC-1,TTTGTCCCATAATCGT-1,TTTGTCTAGCATGTTA-1,TTTGTCTAGGAGGACT-1,TTTGTCTAGTCTATGA-1,TTTGTGGCAGCACGAA-1,TTTGTGGCATCGCTCC-1,TTTGTGTTCACTTCAT-1,TTTGTGTTCATGCGTG-1
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


this one works, above does not

In [22]:
genes_summary <- dt_genes[, rowSums(.SD), .SDcols = -"id"]
names(genes_summary) <- dt_genes$id
genes_summary[1:5]

ENSG00000243485 ENSG00000237613 ENSG00000186092 ENSG00000238009 ENSG00000239945 
              0               0               0              16               1

In [23]:
length(genes_summary)

[1] 36601

In [24]:
atac_summary <- dt_atac[, rowSums(.SD), .SDcols = -"id"]
names(atac_summary) <- dt_atac$id
atac_summary[1:5]

chr1:9782-10672 chr1:180547-181446 chr1:191121-191998 chr1:267553-268447 
                46                 83                 20                 74 
chr1:270906-271782 
                26

# Step 4: Create GenomicRanges

* Convert both the summarized gene expression and peak data into GenomicRanges objects.
* Add the summarized data as metadata to their respective GenomicRanges.

In [25]:
library(GenomicRanges)

In [26]:
unique(features$type)

[1] "Gene Expression" "Peaks"

In [27]:
sum(grepl("Gene Expression", features$type))

[1] 36601

In [28]:
features_dt <- as.data.table(features)

In [29]:
features_genes <- features_dt[grepl("Gene", features_dt$type)]
features_atac <- features_dt[grepl("Peaks", features_dt$type)]

In [30]:
unique(features_genes$chr)

[1] "chr1"       "chr2"       "chr3"       "chr4"       "chr5"      
 [6] "chr6"       "chr7"       "chr8"       "chr9"       "chr10"     
[11] "chr11"      "chr12"      "chr13"      "chr14"      "chr15"     
[16] "chr16"      "chr17"      "chr18"      "chr19"      "chr20"     
[21] "chr21"      "chr22"      "chrX"       "chrY"       NA          
[26] "GL000009.2" "GL000194.1" "GL000195.1" "GL000205.2" "GL000213.1"
[31] "GL000218.1" "GL000219.1" "KI270711.1" "KI270713.1" "KI270721.1"
[36] "KI270726.1" "KI270727.1" "KI270728.1" "KI270731.1" "KI270734.1"

In [31]:
length(features_genes$chr)

[1] 36601

In [32]:
sum(is.na(features_genes$chr))

[1] 13

In [33]:
features_genes$chr[is.na(features_genes$chr)] <- "unknown"

In [34]:
sum(is.na(features_genes$chr))

[1] 0

In [35]:
gr_genes <- GRanges(
    seqnames = features_genes$chr,
    ranges = IRanges(
        start = features_genes$start,
        end = features_genes$end
    ),
    gene_id = dt_genes$id,
    name = features_genes$name,
    sum = genes_summary
)
head(gr_genes)

GRanges object with 6 ranges and 3 metadata columns:
      seqnames        ranges strand |         gene_id        name       sum
         <Rle>     <IRanges>  <Rle> |     <character> <character> <numeric>
  [1]     chr1   29553-30267      * | ENSG00000243485 MIR1302-2HG         0
  [2]     chr1   36080-36081      * | ENSG00000237613     FAM138A         0
  [3]     chr1   65418-69055      * | ENSG00000186092       OR4F5         0
  [4]     chr1 120931-133723      * | ENSG00000238009  AL627309.1        16
  [5]     chr1   91104-91105      * | ENSG00000239945  AL627309.3         1
  [6]     chr1 140338-140339      * | ENSG00000239906  AL627309.2         0
  -------
  seqinfo: 40 sequences from an unspecified genome; no seqlengths

In [36]:
sum(is.na(features_atac$chr))

[1] 0

In [37]:
features_atac$chr[is.na(features_atac$chr)] <- "unknown"

In [38]:
sum(is.na(features_atac$chr))

[1] 0

In [39]:
length(features_atac$chr)
length(features_atac$start)
length(features_atac$end)
length(dt_atac$id)
length(features_atac$id)
length(features_atac$name)

[1] 81156

[1] 81156

[1] 81156

[1] 81156

[1] 81156

[1] 81156

In [40]:
head(features_atac$chr)

[1] "chr1" "chr1" "chr1" "chr1" "chr1" "chr1"

In [44]:
sum(features_atac$id==features_atac$name)

[1] 81156

In [45]:
gr_atac <- GRanges(
    seqnames = features_atac$chr,
    ranges = IRanges(
        start = features_atac$start,
        end = features_atac$end
    ),
    peack_id = features_atac$id,
    sum = atac_summary
)
head(gr_atac)

GRanges object with 6 ranges and 2 metadata columns:
      seqnames        ranges strand |           peack_id       sum
         <Rle>     <IRanges>  <Rle> |        <character> <numeric>
  [1]     chr1    9782-10672      * |    chr1:9782-10672        46
  [2]     chr1 180547-181446      * | chr1:180547-181446        83
  [3]     chr1 191121-191998      * | chr1:191121-191998        20
  [4]     chr1 267553-268447      * | chr1:267553-268447        74
  [5]     chr1 270906-271782      * | chr1:270906-271782        26
  [6]     chr1 273943-274789      * | chr1:273943-274789        25
  -------
  seqinfo: 37 sequences from an unspecified genome; no seqlengths

In [120]:
?GRanges

GRanges-class {GenomicRanges},R Documentation


# Step 5: Gene Annotation for ATACseq data

Using the annotation file Homo_sapiens.GRCh38.114.gtf.gz:
* Create a GenomicRanges object only for protein-coding genes and only for gene features.
* Remap the ATAC-seq GenomicRanges to this object and attach the summarized peak data from step 4.

In [48]:
if (!requireNamespace("rtracklayer", quietly = TRUE)) BiocManager::install("rtracklayer")
library(rtracklayer)

In [51]:
h38 <- rtracklayer::import("Data/Homo_sapiens.GRCh38.114.gtf.gz")

In [55]:
seqnames(h38)

factor-Rle of length 4116048 with 70 runs
  Lengths:     384290     317346     258971 ...         24          4
  Values : 1          2          3          ... KI270718.1 KI270755.1
Levels(70): 1 2 3 4 5 ... KI270750.1 KI270751.1 KI270753.1 KI270755.1

In [52]:
head(h38)

GRanges object with 6 ranges and 22 metadata columns:
      seqnames          ranges strand |         source        type     score
         <Rle>       <IRanges>  <Rle> |       <factor>    <factor> <numeric>
  [1]        1 3069168-3438621      + | ensembl_havana gene               NA
  [2]        1 3069168-3434342      + | havana         transcript         NA
  [3]        1 3069168-3069296      + | havana         exon               NA
  [4]        1 3069260-3069296      + | havana         CDS                NA
  [5]        1 3069260-3069262      + | havana         start_codon        NA
  [6]        1 3186125-3186474      + | havana         exon               NA
          phase         gene_id gene_version   gene_name    gene_source
      <integer>     <character>  <character> <character>    <character>
  [1]      <NA> ENSG00000142611           17      PRDM16 ensembl_havana
  [2]      <NA> ENSG00000142611           17      PRDM16 ensembl_havana
  [3]      <NA> ENSG00000142611           

In [65]:
sum(h38$type == "gene" & h38$gene_biotype == "protein_coding")

[1] 20120

In [75]:
seqlevels(h38) <- ifelse(
    grepl(
        "^([1-9]|1[0-9]|2[0-2]|X|Y)$", 
        seqlevels(h38)
    ), 
    paste0(
        "chr", 
        seqlevels(h38)
    ), 
    seqlevels(h38)
)

In [76]:
seqlevels(h38)

[1] "chr1"       "chr2"       "chr3"       "chr4"       "chr5"      
 [6] "chr6"       "chr7"       "chr8"       "chr9"       "chr10"     
[11] "chr11"      "chr12"      "chr13"      "chr14"      "chr15"     
[16] "chr16"      "chr17"      "chr18"      "chr19"      "chr20"     
[21] "chr21"      "chr22"      "chrX"       "chrY"       "MT"        
[26] "GL000008.2" "GL000009.2" "GL000194.1" "GL000195.1" "GL000205.2"
[31] "GL000213.1" "GL000214.1" "GL000216.2" "GL000218.1" "GL000219.1"
[36] "GL000220.1" "GL000221.1" "GL000224.1" "GL000225.1" "KI270442.1"
[41] "KI270706.1" "KI270710.1" "KI270711.1" "KI270712.1" "KI270713.1"
[46] "KI270714.1" "KI270717.1" "KI270718.1" "KI270719.1" "KI270720.1"
[51] "KI270721.1" "KI270722.1" "KI270726.1" "KI270727.1" "KI270728.1"
[56] "KI270731.1" "KI270733.1" "KI270734.1" "KI270741.1" "KI270742.1"
[61] "KI270743.1" "KI270744.1" "KI270745.1" "KI270746.1" "KI270748.1"
[66] "KI270749.1" "KI270750.1" "KI270751.1" "KI270753.1" "KI270755.1"

In [86]:
h38_protein_coding <- h38[h38$type == "gene" & h38$gene_biotype == "protein_coding"]

In [87]:
seqlevels(h38_protein_coding) == "chr1"

[1]  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[25] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[61] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE

In [88]:
overlap_atac <- findOverlaps(gr_atac, h38_protein_coding)
overlap_atac

Hits object with 58205 hits and 0 metadata columns:
          queryHits subjectHits
          <integer>   <integer>
      [1]        23         725
      [2]        24         725
      [3]        25         725
      [4]        26         679
      [5]        27        1118
      ...       ...         ...
  [58201]     81147       20115
  [58202]     81149       20116
  [58203]     81150       20118
  [58204]     81151       20118
  [58205]     81152       20118
  -------
  queryLength: 81156 / subjectLength: 20120

In [89]:
gr_atac$nearest_gene <- NA
gr_atac$nearest_gene[queryHits(overlap_atac)] <- h38_protein_coding$gene_id[subjectHits(overlap_atac)]

In [92]:
gr_atac[50:70]

GRanges object with 21 ranges and 3 metadata columns:
       seqnames          ranges strand |             peack_id       sum
          <Rle>       <IRanges>  <Rle> |          <character> <numeric>
   [1]     chr1 1143877-1144746      * | chr1:1143877-1144746       664
   [2]     chr1 1157382-1157956      * | chr1:1157382-1157956       705
   [3]     chr1 1171537-1172446      * | chr1:1171537-1172446       271
   [4]     chr1 1173399-1174330      * | chr1:1173399-1174330       482
   [5]     chr1 1182517-1183343      * | chr1:1182517-1183343       136
   ...      ...             ...    ... .                  ...       ...
  [17]     chr1 1291553-1292459      * | chr1:1291553-1292459       193
  [18]     chr1 1305174-1306099      * | chr1:1305174-1306099       795
  [19]     chr1 1307786-1308702      * | chr1:1307786-1308702      4221
  [20]     chr1 1315264-1316182      * | chr1:1315264-1316182       421
  [21]     chr1 1324320-1325215      * | chr1:1324320-1325215      2616
          

# Step 6: Finalize expression data

* Subset the expression GenomicRanges, step 4, to include only protein-coding genes.
* Add gene symbol identifiers to the object.

In [94]:
gr_genes$gene_symbol <- h38_protein_coding$gene_name[match(gr_genes$gene_id, h38_protein_coding$gene_id)]

In [104]:
gr_genes_pr_cod <- gr_genes[!is.na(gr_genes$gene_symbol)]

# Step 7: Data Normalization and Integration

* Normalize both expression and ATAC-seq data using CPM:
    * Divide each column by the column sum, multiply by 106, add a pseudo-count of 1, and apply log2.
* Merge expression and ATAC data based on common genes.
* Provide a summary table of the number of ATAC peaks that could not be merged and a plot of peak intensity distribution chromosome by chromosome. Provide a summary table of the genes which do not show association with ATAC peaks and plot their expression distribution chromosome by chromosome

In [115]:
dt_genes_cmp <- NormalizeData(dt_genes[,-"id"], normalization.method = "LogNormalize", scale.factor = 1e6) %>% as.data.table
dt_genes_cmp[, id := dt_genes$id]
setcolorder(dt_genes_cmp, c("id", setdiff(names(dt_genes_cmp), "id")))

In [117]:
dt_atac_cmp <- NormalizeData(dt_atac[,-"id"], normalization.method = "LogNormalize", scale.factor = 1e6) %>% as.data.table
dt_atac_cmp[, id := dt_atac$id]
setcolorder(dt_atac_cmp, c("id", setdiff(names(dt_atac_cmp), "id")))

In [120]:
merge <- merge(dt_genes_cmp, dt_genes_cmp, by = "id", all = TRUE)

In [123]:
head(dt_genes_cmp)
head(dt_genes_cmp)

id,AAACAGCCAACAGGTG-1,AAACATGCAACAACAA-1,AAACATGCACCTGGTG-1,AAACCAACACAGCCTG-1,AAACCAACAGCAAGAT-1,AAACCAACATTGCGAC-1,AAACCGAAGCACAGCC-1,AAACCGCGTAATTAGC-1,AAACCGCGTCACGAAC-1,⋯,TTTGGTGCATGAGCAG-1,TTTGTCCCAGCTTAGC-1,TTTGTCCCATAATCGT-1,TTTGTCTAGCATGTTA-1,TTTGTCTAGGAGGACT-1,TTTGTCTAGTCTATGA-1,TTTGTGGCAGCACGAA-1,TTTGTGGCATCGCTCC-1,TTTGTGTTCACTTCAT-1,TTTGTGTTCATGCGTG-1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000243485,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000237613,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000186092,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000238009,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000239945,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000239906,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


id,AAACAGCCAACAGGTG-1,AAACATGCAACAACAA-1,AAACATGCACCTGGTG-1,AAACCAACACAGCCTG-1,AAACCAACAGCAAGAT-1,AAACCAACATTGCGAC-1,AAACCGAAGCACAGCC-1,AAACCGCGTAATTAGC-1,AAACCGCGTCACGAAC-1,⋯,TTTGGTGCATGAGCAG-1,TTTGTCCCAGCTTAGC-1,TTTGTCCCATAATCGT-1,TTTGTCTAGCATGTTA-1,TTTGTCTAGGAGGACT-1,TTTGTCTAGTCTATGA-1,TTTGTGGCAGCACGAA-1,TTTGTGGCATCGCTCC-1,TTTGTGTTCACTTCAT-1,TTTGTGTTCATGCGTG-1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000243485,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000237613,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000186092,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000238009,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000239945,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
ENSG00000239906,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [122]:
nrow(merge)
ncol(merge)
head(merge)

[1] 36601

[1] 6019

id,AAACAGCCAACAGGTG-1.x,AAACATGCAACAACAA-1.x,AAACATGCACCTGGTG-1.x,AAACCAACACAGCCTG-1.x,AAACCAACAGCAAGAT-1.x,AAACCAACATTGCGAC-1.x,AAACCGAAGCACAGCC-1.x,AAACCGCGTAATTAGC-1.x,AAACCGCGTCACGAAC-1.x,⋯,TTTGGTGCATGAGCAG-1.y,TTTGTCCCAGCTTAGC-1.y,TTTGTCCCATAATCGT-1.y,TTTGTCTAGCATGTTA-1.y,TTTGTCTAGGAGGACT-1.y,TTTGTCTAGTCTATGA-1.y,TTTGTGGCAGCACGAA-1.y,TTTGTGGCATCGCTCC-1.y,TTTGTGTTCACTTCAT-1.y,TTTGTGTTCATGCGTG-1.y
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,0,0.000000,0,0,0,0,0.000000,0,0,⋯,0.000000,0.000000,0,0.000000,0,0,0,0,0.000000,0
ENSG00000000005,0,0.000000,0,0,0,0,0.000000,0,0,⋯,0.000000,0.000000,0,0.000000,0,0,0,0,0.000000,0
ENSG00000000419,0,0.000000,0,0,0,0,0.000000,0,0,⋯,0.000000,0.000000,0,0.000000,0,0,0,0,5.602846,0
ENSG00000000457,0,0.000000,0,0,0,0,5.455749,0,0,⋯,0.000000,0.000000,0,0.000000,0,0,0,0,0.000000,0
ENSG00000000460,0,0.000000,0,0,0,0,6.146758,0,0,⋯,0.000000,6.099516,0,0.000000,0,0,0,0,0.000000,0
ENSG00000000938,0,6.622155,0,0,0,0,0.000000,0,0,⋯,5.405075,0.000000,0,5.628756,0,0,0,0,5.602846,0


In [ ]:
unmerged_peaks <- atac_peaks_dt[is.na(id)]
unmerged_table <- table(unmerged_peaks$chromosome)

# Step 8: Visualization

* Generate a scatter plot using ggplot2:
    * X-axis: log-transformed expression CPM
    * Y-axis: log-transformed ATAC CPM
* If the plot is too much busy of data divide the plot in the 24 chromosomes